In [1]:
import tensorflow as tf
keras = tf.keras
import numpy as np
from IPython.display import Image, display
from matplotlib import pyplot as plt
from tensorflow.keras.models import load_model

In [7]:
# Cargar tu modelo desde el archivo .h5
model_path = './trained_model.h5'  # Ajusta esta ruta a donde está tu archivo .h5
model = load_model(model_path)

In [8]:
# Nombre de la última capa convolucional del modelo
last_conv_layer_name = 'conv_layer_2' 

In [9]:
# Crear el modelo grad_model que mapea las activaciones de la última capa convolucional y la salida
grad_model = keras.models.Model(
    model.inputs, [model.get_layer(last_conv_layer_name).output, model.output]
)

ValueError: The layer sequential has never been called and thus has no defined output.